In [2]:
import os 
import glob
import pandas as pd
from ics import Calendar, Event
from dataclasses import dataclass
import numpy as np

In [3]:
import logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


In [4]:
base_folder = os.getcwd()
source_folder = base_folder + '/healthlake/'
apple_health_files = glob.glob(source_folder + '*.json')

names = [
    'apple_exercise_time',
    'active_energy',
    'calories_burnt',
    'calories_consumed',
    'carbohydrates',
    'dietary_caffeine',
    'dietary_energy',
    'dietary_sugar',
    'fiber',
    'protein',
    'sleep_analysis',
    'total_fat',
    'weight_body_mass'
]

cols = [
    'qty',
    'dates',
    'name',
    'units'
]
df_raw = pd.DataFrame()

for json_file in apple_health_files:
    json_raw = pd.read_json(json_file, lines = True)
    df_raw = pd.concat([df_raw, json_raw])

In [117]:
@dataclass 
class AppleHealthEvent(Event):
    """
    An event derived from Apple Health data
    For usage within .ics format 
    """
    date: str
    description: str 
    title: str
            
    @property 
    def event(self):
        all_day_date = f"{self.date} 00:00:00"
        e = Event()
        e.name = self.title
        e.description = self.description
        e.begin = all_day_date
        e.end = all_day_date
        e.make_all_day()

        return e

In [122]:
@dataclass 
class HealthStat: 
    """
    A health stat derived from Apple Health data
    """
    name: str
    qty: float
    units: str

In [123]:
from typing import List
@dataclass 
class HealthEvent: 
    """
    An event derived from Apple Health data
    For usage within .ics format 
    """
    type: str 
    date: str 
    stats: List[HealthStat]

    @property
    def title(self) -> str: 
        if type == 'sleep': 
            


In [201]:
@dataclass
class Time: 
    "A basic time object"
    time: float     
        
    @property 
    def hours(self) -> float: 
        hours = int(self.time)
        return hours 
    
    @property
    def minutes(self) -> str:
        minutes = int((self.time - self.hours) * 60)
        return minutes
    
    @property 
    def title(self) -> str: 
        title = f"{self.hours}h {self.minutes}m"
        return title


@dataclass 
class Food: 
    "A basic food object"
    carbohydrates: float 
    protein: float 
    total_fat: float 
    fiber: float 
    
    def __post__init(self): 
        # rename objects for easier usage 
        self.carb = self.carbohydrates
        self.fat = self.total_fat
    
    @property 
    def calories(self) -> float: 
        calories = (self.carb + self.protein) * 4 + (self.fat) * 9
        return calories
    
    @property 
    def title(self) -> str: 
        title = f"🔥 {self.calories:.0f} cals ({self.carb:.0f}C, {self.protein:.0f}P, {self.fat:.0f}F)"
        return title 
    
    @property 
    def description(self) -> str: 
        description = f"""
        🔥 {self.calories:.0f} kcal
        🥞 {self.title} 
        🍇 {self.fiber:.0f}
        """
        return description

@dataclass
class Sleep:
    "A basic sleep object"
    asleep: Time
    inBed: Time
    inBedStart: str

    def __post__init__(self): 
        # rename objects for easier usage
        self.time_asleep = self.asleep
        self.time_in_bed = self.inBed
        self.in_bed_time = self.inBedStart
    
    @property 
    def efficiency(self) -> float: 
        efficiency = self.time_asleep.time / self.time_in_bed.time * 100
        return efficiency
    
    @property
    def efficiency_title(self) -> str: 
        efficient = f"{self.efficiency:.0f}%"
        efficiency_title = f"🛏️ {efficient}"
        return efficiency_title
    
    @property 
    def title(self) -> str: 
        title = f"💤 {self.time_asleep.title} ({self.in_bed_time})"
        return title 
    
    @property
    def description(self) -> str: 
        description = f"""
        💤 Time asleep: {self.time_asleep.title}
        🛏️ Time in bed: {self.time_in_bed.title}
        🧮 Efficiency: {self.efficiency_title}
        """
        return description
    
    
@dataclass
class Activity: 
    "A basic activity for activity and mindfulness"
    activity_mins: Time 
    mindful_mins: Time 
    
    @property 
    def activity_title(self) -> str: 
        title = f"{self.activity_mins.title} active" 
        return title
    
    @property 
    def mindful_title(self) -> str:
        title = f"{self.mindful_mins.minutes} mins mindful"
        return title
    
    @property 
    def title(self) -> str: 
        title = f"🧠 {self.mindful_title}"
        
    @property 
    def description(self) -> str: 
        description = f"""
        🚴‍♂️ Activity: {self.activity_title}
        🧘 Mindful: {self.mindful_title}
        """
        return description

In [8]:
def convert_kj_to_cal(row, new_name):
    """Converts kj to calories"""
    row_dict = row.to_dict()
    calorie_value = int(row['qty']/4)
    row_dict['qty'] = calorie_value
    row_dict['name'] = new_name
    row_dict['units'] = 'kcal'
    
    return pd.DataFrame(row_dict, index=[0])


In [151]:
# config driven
sleep_stats_cols = ['asleep', 'inBed', 'inBedStart']
# ETL transformations 

df_ahc = df_raw.copy() 

# define transformations to go from df_raw to df_ahc (apple-health-calendar)
# cleaning values 
df_ahc['dates'] = pd.to_datetime(df_ahc['date']).dt.date

# create sleep stats
sleep_stats_cols.insert(0, 'dates')

daily_sleep = df_ahc.query('name == "sleep_analysis" \
    and sleepSource == "AutoSleep"')[sleep_stats_cols]

daily_sleep['inBedStart'] = pd.to_datetime(daily_sleep['inBedStart']).dt.time
daily_sleep['name'] = 'sleep'

# # create calories 
active_energy_rows = df_ahc[df_ahc['name'] == 'active_energy'][cols]
dietary_energy_rows = df_ahc[df_ahc['name'] == 'dietary_energy'][cols]

for _, row in active_energy_rows.iterrows():
    df_row = convert_kj_to_cal(row, 'calories_burnt')    
    df_ahc = pd.concat([df_ahc, df_row])
    
for _, row in dietary_energy_rows.iterrows(): 
    df_row = convert_kj_to_cal(row, 'calories_consumed')
    df_ahc = pd.concat([df_ahc, df_row])
    
    
# filter out values 

for _, row in df_ahc.iterrows(): 
    col_name = row['name']
    event_type = column_event_types.get(col_name)
    df_ahc['type'] = event_type

ahc = df_ahc[~df_ahc['type'].isna()].copy()[cols]
ahc = ahc.reset_index(drop=True)
ahc['qty']  =ahc['qty'].round(2)


# round values 
# df_ahc['qty'] = df_ahc['qty'].round(2)


In [152]:
ahc

,qty,dates,name,units
0,3039.92,2022-09-10,active_energy,kJ
1,3553.97,2022-09-11,active_energy,kJ
2,3627.76,2022-09-12,active_energy,kJ
3,3750.79,2022-09-13,active_energy,kJ
4,4264.28,2022-09-14,active_energy,kJ
...,...,...,...,...
390,2012.00,2022-09-12,calories_consumed,kcal
391,2057.00,2022-09-13,calories_consumed,kcal
392,2726.00,2022-09-14,calories_consumed,kcal
393,2665.00,2022-09-15,calories_consumed,kcal


In [20]:
def generate_calendar(df):
    """
    Generates a CSV and ICS from the dataframe
    :param df: cleansed dataframe from `create_description_cols`
    :param outputs: as type string - a combination of both the local and public storage
    """

    # output_csv_path = f"{output_path}/{file_name}.csv"
    # calendar_file_name = f'{file_name}.ics'
    file_name = 'apple_health'

    csv_file_name = f"{file_name}.csv"
    ics_file_name  = f"{file_name}.ics"


    LOGGER.info("Generating calendar (as .ICS)")
    c = Calendar()
    for _, row in df.iterrows():
        e = create_event(row['date'], row['name'], row['dsc'])
        c.events.add(e)

    df.to_csv(csv_file_name, index=False)

    with open(ics_file_name, 'w') as f:
        f.write(str(c))
        f.close()

    LOGGER.info("Outputing CSV and ICS to: %s", csv_file_name)
    return

In [136]:
event_types = {
    "food": ['carbohydrates', 'protein', 'total_fat', 'fiber'],
    "activity": ['calories_burnt', 'calories_consumed'],
    "sleep": ['asleep', 'inBed', 'inBedStart']
}

In [138]:
column_event_types = dict((col_name, types) for types, cols in event_types.items() for col_name in cols)

In [102]:
# config driven 
food_cols = ['carbohydrates', 'protein', 'total_fat', 'fiber']
activity_cols = ['calories_burnt', 'calories_consumed']

foods = df_ahc[df_ahc['name'].isin(food_cols)]
activities = df_ahc[df_ahc['name'].isin(activity_cols)]

daily_food = foods.pivot(index = 'dates', columns = 'name')['qty'].reset_index()
daily_activity = activities.pivot(index = 'dates', columns = 'name')['qty'].reset_index()

# add event column and functionise description 
daily_food['name'] = 'food'
daily_activity['name'] = 'activity' 


daily_events = [daily_sleep, daily_food, daily_activity]
        

In [133]:
for _, row in df_ahc.iterrows(): 
    if row['name'] == 'sleep': 
        sleep = Sleep(time_asleep = Time(row['asleep']), time_in_bed = Time(row['inBed']), in_bed_time=row['inBedStart'])
        row['dsc'] = sleep.description
    elif row['name'] == 'food': 
        food = Food(carbs = row['carbohydrates'], protein = row['protein'], fat = row['total_fat'], fiber = row['fiber'])
        row['dsc'] = food.description
    elif row['name'] == 'activity': 
        activity = Activity(activity_mins = Time(row['calories_burnt']), mindful_mins = Time(row['calories_consumed']))
        row['dsc'] = activity.description
    

,qty,dates,name,units
0,3039.92,2022-09-10,active_energy,kJ
1,3553.97,2022-09-11,active_energy,kJ
2,3627.76,2022-09-12,active_energy,kJ
3,3750.79,2022-09-13,active_energy,kJ
4,4264.28,2022-09-14,active_energy,kJ
...,...,...,...,...
76,2012.00,2022-09-12,calories_consumed,kcal
77,2057.00,2022-09-13,calories_consumed,kcal
78,2726.00,2022-09-14,calories_consumed,kcal
79,2665.00,2022-09-15,calories_consumed,kcal


In [132]:
c = Calendar()
for _, day in daily_sleep.iterrows(): 
    
    day_sleep = Sleep(
        time_asleep = Time(day['asleep']),
        time_in_bed = Time(day['inBed']),
        in_bed_time = day['inBedStart']
    )
    
    # create event from sleep event 
    e = AppleHealthEvent(
        date = day['dates'],
        title = day_sleep.title,
        description = day_sleep.description
    ).event

    c.events.add(e)

for _, day in daily_food.iterrows(): 
    day_food = Food(
        carbs = day['carbohydrates'],
        protein = day['protein'],
        fat = day['total_fat'],
        fiber = day['fiber']
    )
    
    # create event from sleep event 
    e = AppleHealthEvent(
        date = day['dates'],
        title = day_food.title,
        description = day_food.description
    ).event

    c.events.add(e)

In [202]:
available_dates = ahc['dates'].unique()
for date in available_dates:
    daily_stats = ahc[ahc['dates'] == date]
    for types, col_names in event_types.items(): 
        # collect object name and arguments
        dataclass_name = types.capitalize()
        dataclass_obj = globals()[dataclass_name]
        obj_args = daily_stats[daily_stats['name'].isin(col_names)][['name', 'qty']]
        obj_args = dict(obj_args.values)
        # print(dict(obj_args.values))
        if obj_args: 
            obj = dataclass_obj(**obj_args)
            print(obj)
        


Food(carbohydrates=184.15, protein=184.59, total_fat=51.16, fiber=26.6)


TypeError: Activity.__init__() got an unexpected keyword argument 'calories_burnt'

Food
<class '__main__.Food'>
Activity
<class '__main__.Activity'>
Sleep
<class '__main__.Sleep'>


In [49]:
def make_event_details(row, self.type): 
    """Create a description field for the event"""

    if self.type in ('sleep'): 
        time = row['qty']
        hours = int(time)
        minutes = int((time - hours) * 60)
        value = f"{hours} hours {minutes} mins"
        summary = f"{hours}h {minutes}m"
    
    elif self.type in ('food'): 
        carbs, protein, fats, fiber = row['carbohydrates'], row['protein'], row['total_fat'], row['fiber']
        summary = f"{protein:.0f} P / {carbs:.0f} C / {fats:.0f} F"
        value = f"""
        Macros: {summary}
        Fiber: {fiber:.0f}
        Calories: {row['calories_consumed']:.0f} kcal
        """
        
    
    elif self.type in ('activity'): 
        burned, consumed = row['calories_burnt'], row['calories_consumed']
        value = f"{burned:.0f} kcal / {consumed:.0f} kcal"
        summary = f"{burned:.0f} kcal"
        # summary = f"{row['mindful_minutes']:.0f} mins"
    
    title = f"{_collect_event_emoticon(self.type)} {summary}"
    description = f"{_collect_event_emoticon(self.type)} {value}"
    
    return title, description
LOGGER.info("Generating calendar (as .ICS)")
c = Calendar()
for event_df in daily_events: 
    for _, event_row in event_df.iterrows(): 
        self.type =  event_df['name'].iloc[0]
        summary, description = make_event_details(event_row, self.type)
        # print(description)
        date = event_row['dates']
        e = create_event(date, summary, description)
        c.events.add(e)

INFO:__main__:Generating calendar (as .ICS)


In [50]:
with open('apple_health.ics', 'w') as f:
    f.write(str(c))
    f.close()